## Fine Tuning T5 model with Azure ML using Azure Container for PyTorch 

This tutorial shows how to fine tune the T5 model to generate a summary of a news article. We then deploy it to an online endpoint for real time inference. The model is trained on a tiny sample of the dataset with a small number of epochs to illustrate the fine tuning approach.

### Learning Objectives
- Fine tune the T5 small model for the `Summarization` task with `Azure ML` 
- Leverage the `ACPT` environment with state of art accelerators
- Increase training efficiency using [`DeepSpeed`](https://github.com/microsoft/DeepSpeed) and [`ONNX Runtime`](https://github.com/microsoft/onnxruntime)
- Model Evaluation uring prebuilt component
- Register the model with AzureML
- Deploy and inference using MIR and ONNX Runtime


###### T5-small is a 60 million parameter model based on text-to-text framework and is used for several NLP tasks, including machine translation, document summarization, question answering pretrained on Colossal Clean Crawled Corpus (C4) dataset.

translation (green), linguistic acceptability (red), sentence similarity (yellow), and document summarization (blue)

##### In this workshop, we will be fine tuning the document summarization task.


![Image](assets/t5modelcard.PNG)

#### 1. Prerequisites to install Azure ML Python SDK Version 2 
Please restart kernel after pip installs to sync environment with new modules.

In [ ]:
%pip install azure-ai-ml azure-identity datasets azure-cli
%pip install onnxruntime==1.15.1 transformers==4.29.2 torch==2.0

In [ ]:
# !az login --use-device-code

#### 2. Connect to Azure Machine Learning workspace

Before we dive in the code, you'll need to connect to your workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

For this lab, we've already setup an AzureML Workspace for you. If you'd like to learn more about `Workspace`s, please reference [`AzureML's documentation`](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?view=azureml-api-2&tabs=azure-portal).

We are using the `DefaultAzureCredential` to get access to workspace. `DefaultAzureCredential` should be capable of handling most scenarios. If you want to learn more about other available credentials, go to [`Set up authentication`](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk&view=azureml-api-2) for more available credentials.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential

credential = AzureCliCredential()
credential.get_token("https://management.azure.com/.default")

ml_client = MLClient(
    credential,
    subscription_id="ed2cab61-14cc-4fb3-ac23-d72609214cfd",
    resource_group_name="AMLDataCache",
    workspace_name="datacachetest",
)

#### 3. Create a compute

Azure Machine Learning needs a compute resource to run a job. This resource can be single or multi-node machines with Linux or Windows OS. In the following example script, we provision a `Standard_ND40rs_v2` SKU which is infiniband enabled to provide higher node communication bandwidth and low latency with mellanox drivers to create an Azure Machine Learning compute. You can get the list and more detail [here](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-hpc#rdma-capable-instances)

In [ ]:
from azure.ai.ml.entities import AmlCompute

experiment_name = "T5-Summarization-news-summary"

# If you already have a gpu cluster, mention it here. Else will create a new one
compute_cluster = "v100"
try:
    compute = ml_client.compute.get(compute_cluster)
    print("successfully fetched compute:", compute.name)
except Exception as ex:
    print("failed to fetch compute:", compute_cluster)
    print("creating new Standard_ND40rs_v2 compute")
    compute = AmlCompute(
        name=compute_cluster,
        size="Standard_ND40rs_v2", # Info on Standard_ND40rs_v2 SKU: https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series
        min_instances=1,
        max_instances=2,  # For multi node training set this to an integer value more than 1
    )
    ml_client.compute.begin_create_or_update(compute).wait()
    print("successfully created compute:", compute.name)


#### 4. Create a job environment using Azure Container for Pytorch

We will be creating a custom environment using existing ACPT curated environment consisting of state of art technologies like Deepspeed, OnnxRuntime. You can get more detail from [Custom Environment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-azure-container-for-pytorch-environment?view=azureml-api-2)


view the [Environments in Azure Machine Learning studio](https://ml.azure.com/environments)

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext
import datetime

# Define an environment name
env_name = "env-" + datetime.datetime.now().strftime("%m%d%H%M%f")

env_docker_context = Environment(
    build=BuildContext(path="src/Environment/context"),
    name=env_name,
    description="Environment created from a Docker context.",
)
ml_client.environments.create_or_update(env_docker_context)


#### 5. Pick the dataset for fine-tuning the model

The [CNN DailyMail](https://huggingface.co/datasets/cnn_dailymail) dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. It is larger than 1GB when uncompressed. 

We want this sample to run quickly, so a copy of the fraction of dataset is used for fine tuning job.This means the fine tuned model will have lower accuracy, hence it should not be put to real-world use. 
* Visualize some data rows. 

In [ ]:
import pandas as pd
pd.set_option(
    "display.max_colwidth", 1000
)

train_df = pd.read_json("./src/Finetune/cnn_daily.jsonl", lines=True)
train_df.head()

#### 6. Finetune the T5 small model for Summarization task

Leveraging Deepspeed and Onnxruntime accelarators for improving the efficiency for memory and compute and in turn reduce the training cost. 

The table below details some of the parameters passed to the training job.

| Parameters/accelarators | Description |
| ----------------- | --- |
| model_name | The name of the model getting finetuned. Here we specify T5-small. |
| ort | [Onnx runtime](https://github.com/microsoft/onnxruntime) accelarates 2x speed up in training time for SOTA models and optimizes memory to fit larger model such as GPT3 on 16GB GPU which would otherwise run out of mmemory. |
| deepspeed | [Deepspeed](https://github.com/microsoft/deepspeed) enables running billions of parameter models distributed across GPUs and provide different stages for memory and compute efficiency. |
| number of epochs | 1 |
| max train samples | 10 |
| Nebula | checkpointing |


In [ ]:
from azure.ai.ml import command

env_name = "MSBuildLab110_env@latest" # FOR DEMO

job = command(
    code="src/Finetune/",
    command="python train_summarization_optimum.py \
        --deepspeed ds_config.json \
        --model_name_or_path t5-small \
        --dataset_name cnn_dailymail \
        --max_train_samples=10 \
        --max_eval_samples=10 \
        --dataset_config '3.0.0' \
        --do_train \
        --num_train_epochs=1 \
        --per_device_train_batch_size=16 \
        --per_device_eval_batch_size=16  \
        --output_dir outputs \
        --overwrite_output_dir \
        --fp16 \
        --optim adamw_ort_fused",
    compute=compute_cluster,
    environment=env_name,
    instance_count=1,  
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 8,
    },
)
job = ml_client.jobs.create_or_update(job)
job.studio_url

#### Results show **~300%** improvement of Fine-tune job with 100 epoch and CNN_Daily dataset with ORT, Deepspeed and Nebula checkpointing

| Accelerator | Train Runtime | Train Samples Per Sec | Train Steps Per Sec | Train Loss | FLOPS          | AML Link                                                   |
| ----------- | ------------- | --------------------- | ------------------- | ---------- | -------------- | ---------------------------------------------------------- |
| -           | 1d 9h 6m 22s  | 241.782               | 0.945               | 1.704454   | 7.7719 x 10^18 | [100epoch_and_no_accelarator][100epoch_and_no_accelarator] |
| ORT+DS      | 10h 53 m 36s  | 744.384               | 2.909               | 1.850196   | 7.8759 x 10^18 | [100epoch_with_DS_ORT_Nebula][100epoch_with_DS_ORT_Nebula] |

[100epoch_and_no_accelarator]: https://ml.azure.com/experiments/id/236409a2-f1d9-41da-9b7e-19ec7bfd23a6/runs/tough_cord_3wz4fymfjf?wsid=/subscriptions/ed2cab61-14cc-4fb3-ac23-d72609214cfd/resourcegroups/AMLDataCache/workspaces/datacachetest&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
[100epoch_with_DS_ORT_Nebula]: https://ml.azure.com/experiments/id/236409a2-f1d9-41da-9b7e-19ec7bfd23a6/runs/tough_fennel_7n42y51slk?wsid=/subscriptions/ed2cab61-14cc-4fb3-ac23-d72609214cfd/resourcegroups/AMLDataCache/workspaces/datacachetest&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

#### 7. Operationalizing the model

##### 7.1 Register Onnx model
**NOTE: STEP 6 (T5 FINETUNE) MUST COMPLETE BEFORE RUNNING THIS CELL**

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Model
import time

timestamp = str(int(time.time()))
model_name = "T5Model"
job.name = "loyal_nose_p0r4vzq8w8" # FOR DEMO

#Onnx model registration
modelpath = "azureml://jobs/{jobname}/outputs/artifacts/outputs/onnx".format(jobname = job.name)
cloud_model = Model(
    path=modelpath,
    name=model_name+"_onnx",
    type=AssetTypes.CUSTOM_MODEL,
    description="Model created from cloud path.",
    version=timestamp,
)
ml_client.models.create_or_update(cloud_model)

##### 7.2 Create online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

# Define an endpoint name
endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a endpoint for T5 summarization model",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
ml_client.begin_create_or_update(endpoint).result()

##### 7.3 Deploy scoring file to the endpoint

In [ ]:
from azure.ai.ml.entities import (
    CodeConfiguration,
    ManagedOnlineDeployment
)

model = ml_client.models.get(name=model_name+"_onnx", version = timestamp)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment="MSBuildLab110_env@latest",
    code_configuration=CodeConfiguration(
        code=".", scoring_script="src/Operationalize/score_onnx.py"
    ),
    instance_type="Standard_F8s_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(blue_deployment)

##### 7.4: Invoke the endpoint to score data by using your model
**NOTE: STEP 7.3 (ENDPOINT DEPLOYMENT) MUST COMPLETE BEFORE RUNNING THIS CELL**

Test the blue deployment with some sample data


In [ ]:
endpoint_name = "MSBuildLab110-endpoint" # FOR DEMO

ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="src/Operationalize/payload.json",
)

##### 7.5: Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)

#### Aside: Scoring files for ONNX Runtime Inference vs. Hugging Face Inference

![Image](assets/T5_beamsearch.png)

In [ ]:
import json 
import numpy as np
import onnxruntime
onnxruntime.set_default_logger_severity(3)
import os
import time
from transformers import AutoTokenizer

# Documentation: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints
# Troubleshooting: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints
  
# The init() method is called once, when the web service starts up.
def init():  
    global SESS
    global TOKENIZER
    # The AZUREML_MODEL_DIR environment variable indicates  
    # a directory containing the model file you registered.  
    # model_filename = os.path.join(os.environ['AZUREML_MODEL_DIR'], "onnx/outputs_beam_search.onnx")  

    model_filename = "src/Model/onnx/outputs_beam_search.onnx" 
    SESS = onnxruntime.InferenceSession(model_filename, providers=["CPUExecutionProvider"])

    TOKENIZER = AutoTokenizer.from_pretrained("t5-small")
  
# The run() method is called each time a request is made to the scoring API.  
def run(data):
    json_data = json.loads(data)
    input_data = json_data["inputs"]["article"]
    
    input_ids = TOKENIZER(str(input_data), return_tensors="pt").input_ids

    ort_inputs = {
        "input_ids": np.array(input_ids, dtype=np.int32),
        "max_length": np.array([512], dtype=np.int32),
        "min_length": np.array([0], dtype=np.int32),
        "num_beams": np.array([1], dtype=np.int32),
        "num_return_sequences": np.array([1], dtype=np.int32),
        "length_penalty": np.array([1.0], dtype=np.float32),
        "repetition_penalty": np.array([1.0], dtype=np.float32)
    }
    
    out = SESS.run(None, ort_inputs)[0][0] # 0th batch, 0th sample

    summary = TOKENIZER.decode(out[0], skip_special_tokens=True)

    # You can return any JSON-serializable object.
    return {"summary": summary}

def test():
    # NOTE: You need to comment out model_filename = os.path.join(...) in init() for local testing
    init()
    payload = {
        "inputs": {
            "article": ["summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."],
            "params": {
                "max_new_tokens": 512
            }
        }
    }
    payload = str.encode(json.dumps(payload))
    res = run(payload)
    print(res)

    # timed run
    start = time.time()
    for i in range(10):
        _ = run(payload)
    diff = time.time() - start
    print(f"time {diff/10} sec")

test()


In [ ]:
import numpy as np
import os
from transformers import pipeline
import json 
import time
import joblib
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForSeq2SeqLM
import torch

# Documentation: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints
# Troubleshooting: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints
  
# The init() method is called once, when the web service starts up.
def init():  
    global MODEL
    global TOKENIZER
    # The AZUREML_MODEL_DIR environment variable indicates  
    # a directory containing the model file you registered.  
    # model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'])
    # model_file = os.path.join(os.environ['AZUREML_MODEL_DIR'], "pytorch_model.bin")

    model_path = "src/Model"
    model_file = "src/Model/pytorch_model.bin"
    TOKENIZER = AutoTokenizer.from_pretrained(model_path)
    config = AutoConfig.from_pretrained(model_path)
    MODEL = AutoModelForSeq2SeqLM.from_pretrained(model_file, config=config) 
    
  
# The run() method is called each time a request is made to the scoring API.  
def run(data):
    json_data = json.loads(data)
    input_data = json_data["inputs"]["article"]
    inputs = TOKENIZER(str(input_data), return_tensors="pt").input_ids

    out = MODEL.generate(inputs, max_new_tokens=512, do_sample=False)

    summary = TOKENIZER.decode(out[0], skip_special_tokens=True)
      
    # You can return any JSON-serializable object.  
    return {"summary": summary}

    
def test():
    # NOTE: You need to comment out model_file/path = os.path.join(...) in init() for local testing
    init()
    payload = {
        "inputs": {
            "article": ["summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."],
            "params": {
                "max_new_tokens": 512
            }
        }
    }
    payload = str.encode(json.dumps(payload))
    res = run(payload)
    print(res)
    
    # timed run
    start = time.time()
    for i in range(10):
        _ = run(payload)
    diff = time.time() - start
    print(f"time {diff/10} sec")

test()